In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
model = AutoModel.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
model.eval()

In [ ]:
inputs = tokenizer(df_pivote['standardized_smiles'].values.tolist(), return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    outputs = model(**inputs)

embeddings = outputs.last_hidden_state.mean(dim=1)

print("Shape:", embeddings.shape)

In [ ]:
data_tsne_compounds = TSNE(random_state=42).fit_transform(embeddings)

df_tsne_compounds = pd.DataFrame(data=data_tsne_compounds, columns=['t-SNE 1', 't-SNE 2'])
